In [1]:
import psycopg as pg
from pgvector.psycopg import register_vector
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.decomposition import TruncatedSVD
import numpy as np
from connections import dbname, user, password, host, port

In [2]:
textos = pd.read_csv('textos.csv')

In [3]:
textos

,AUTOR,TITULO,TEXTO
0,Miguel de Cervantes Saavedra,Quijote - Primera Parte,El ingenioso hidalgo don Quijote de la Mancha\...
1,Miguel de Cervantes Saavedra,Quijote - Segunda Parte,Segunda parte del ingenioso caballero don Quij...
2,Garcilaso de la Vega,ÉGLOGA PRIMERA,"El dulce lamentar de dos pastores,\nSalicio ju..."
3,Garcilaso de la Vega,ÉGLOGA SEGUNDA,ALBANIO\n\nEn medio del invierno está templada...
4,Garcilaso de la Vega,ÉGLOGA TERCERA,"Aquella voluntad honesta y pura,\nilustre y he..."
...,...,...,...
127,Benito Pérez Galdós,España trágica,- I -\n«1.º de Enero.- Ha sonado la última cam...
128,Benito Pérez Galdós,Amadeo I,- I -\nEl 2 de Enero de 1871 vimos entrar en l...
129,Benito Pérez Galdós,La Primera República,"- I -\nVenid acá otra vez, fieles parroquianos..."
130,Benito Pérez Galdós,De Cartago a Sagunto,"- I -\nArriba otra vez, arriba, Tito pequeñín ..."


In [4]:
vectorizer = TfidfVectorizer()
emb = vectorizer.fit_transform(textos['TEXTO'])

In [5]:
%%time
trunc = TruncatedSVD(128)
embr = trunc.fit_transform(emb)

CPU times: total: 20.1 s
Wall time: 6.43 s


In [6]:
textos['emb'] = [vec.tolist() for vec in embr]
textos['emb']

0      [0.9476357306492846, 0.06068554806194861, -0.0...
1      [0.935076278870549, 0.03510169759942742, -0.03...
2      [0.8349001953267455, 0.18602241097910566, 0.16...
3      [0.8787824372138426, 0.14469031439018895, 0.14...
4      [0.8343033820264676, 0.008772285059167436, 0.2...
                             ...                        
127    [0.9688810912619952, -0.12942458429824857, 0.0...
128    [0.9765492402505992, -0.07754434140534049, 0.0...
129    [0.9705626227420431, -0.09337538083696167, 0.0...
130    [0.9715119134897204, -0.09384456196995013, 0.0...
131    [0.9668054889319802, -0.10776291344748438, 0.0...
Name: emb, Length: 132, dtype: object

In [7]:
conn = pg.connect(dbname= dbname, user= user, password= password, host= host, port= port)
conn.autocommit = True

In [8]:
cursor = conn.cursor()
sql = '''DROP database if exists vector'''
cursor.execute(sql)

sql = '''CREATE database vector'''
cursor.execute(sql)


<psycopg.Cursor [COMMAND_OK] [IDLE] (host=127.0.0.1 database=postgres) at 0x1d140983ed0>

In [9]:
conn = pg.connect(dbname= 'vector', user= user, password= password, host= host, port= port)

In [10]:
%%time
conn.execute('CREATE EXTENSION IF NOT EXISTS vector') #la instalación en windows es una lata, instalar VSTUDIO 2022

CPU times: total: 0 ns
Wall time: 25.9 ms


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=127.0.0.1 user=postgres database=vector) at 0x1d140983e20>

In [11]:
register_vector(conn)

In [12]:
textos = textos.drop(columns = 'TEXTO').reset_index(drop = True)

In [13]:
textos

,AUTOR,TITULO,emb
0,Miguel de Cervantes Saavedra,Quijote - Primera Parte,"[0.9476357306492846, 0.06068554806194861, -0.0..."
1,Miguel de Cervantes Saavedra,Quijote - Segunda Parte,"[0.935076278870549, 0.03510169759942742, -0.03..."
2,Garcilaso de la Vega,ÉGLOGA PRIMERA,"[0.8349001953267455, 0.18602241097910566, 0.16..."
3,Garcilaso de la Vega,ÉGLOGA SEGUNDA,"[0.8787824372138426, 0.14469031439018895, 0.14..."
4,Garcilaso de la Vega,ÉGLOGA TERCERA,"[0.8343033820264676, 0.008772285059167436, 0.2..."
...,...,...,...
127,Benito Pérez Galdós,España trágica,"[0.9688810912619952, -0.12942458429824857, 0.0..."
128,Benito Pérez Galdós,Amadeo I,"[0.9765492402505992, -0.07754434140534049, 0.0..."
129,Benito Pérez Galdós,La Primera República,"[0.9705626227420431, -0.09337538083696167, 0.0..."
130,Benito Pérez Galdós,De Cartago a Sagunto,"[0.9715119134897204, -0.09384456196995013, 0.0..."


In [14]:

table_create_command = """
CREATE TABLE embeddings (
            id SERIAL primary key, 
            author text,
            title text,
            embedding vector(128)
            );
            """
cur = conn.cursor()
cur.execute(table_create_command)
cur.close()
conn.commit()

In [15]:
textos

,AUTOR,TITULO,emb
0,Miguel de Cervantes Saavedra,Quijote - Primera Parte,"[0.9476357306492846, 0.06068554806194861, -0.0..."
1,Miguel de Cervantes Saavedra,Quijote - Segunda Parte,"[0.935076278870549, 0.03510169759942742, -0.03..."
2,Garcilaso de la Vega,ÉGLOGA PRIMERA,"[0.8349001953267455, 0.18602241097910566, 0.16..."
3,Garcilaso de la Vega,ÉGLOGA SEGUNDA,"[0.8787824372138426, 0.14469031439018895, 0.14..."
4,Garcilaso de la Vega,ÉGLOGA TERCERA,"[0.8343033820264676, 0.008772285059167436, 0.2..."
...,...,...,...
127,Benito Pérez Galdós,España trágica,"[0.9688810912619952, -0.12942458429824857, 0.0..."
128,Benito Pérez Galdós,Amadeo I,"[0.9765492402505992, -0.07754434140534049, 0.0..."
129,Benito Pérez Galdós,La Primera República,"[0.9705626227420431, -0.09337538083696167, 0.0..."
130,Benito Pérez Galdós,De Cartago a Sagunto,"[0.9715119134897204, -0.09384456196995013, 0.0..."


In [16]:
%%time
 #agregar datos en pg
register_vector(conn)
cur = conn.cursor()

data_list = [(row['AUTOR'], row['TITULO'], np.array(row['emb'])) for index, row in textos.iterrows()]

cur.executemany("INSERT INTO embeddings (author, title, embedding) VALUES (%s, %s, %s)", data_list)



conn.commit()
cur.close()

CPU times: total: 15.6 ms
Wall time: 16 ms


In [17]:
data_list

[('Miguel de Cervantes Saavedra',
  'Quijote - Primera Parte',
  array([ 9.47635731e-01,  6.06855481e-02, -4.49487585e-02, -9.88106348e-02,
          4.67204983e-02,  3.05899092e-02, -3.94587688e-03, -3.16662364e-02,
         -4.53656248e-02, -2.99422187e-02, -5.88948488e-03,  1.66741823e-02,
         -1.58854592e-02,  2.15967900e-02,  2.99462954e-02, -4.96855151e-03,
          5.92889253e-03,  9.10296397e-03, -1.45211465e-02,  6.03667407e-03,
          3.20857373e-03, -9.05611234e-04,  2.38791901e-02, -8.75011195e-03,
         -4.48020482e-03, -7.82055048e-03, -2.93433219e-03,  1.40361734e-02,
          2.49557414e-02,  7.29052352e-03,  6.43176115e-03, -1.88305075e-02,
         -4.05647118e-03,  7.46581699e-03, -1.70839302e-02, -1.23933913e-02,
         -8.27531615e-03, -2.28865009e-03,  6.20915442e-03, -9.93421453e-04,
         -1.67710498e-02, -1.44265969e-02,  2.98524111e-03, -2.70770050e-02,
         -2.15045434e-03,  4.88113983e-03, -9.19720214e-03,  1.02331016e-02,
         -2.3

In [18]:
cur = conn.cursor()
cur.execute('SELECT * FROM embeddings')

<psycopg.Cursor [TUPLES_OK] [INTRANS] (host=127.0.0.1 user=postgres database=vector) at 0x1d146bbc040>

In [19]:
cur.fetchone()

(1,
 'Miguel de Cervantes Saavedra',
 'Quijote - Primera Parte',
 array([ 9.4763571e-01,  6.0685549e-02, -4.4948757e-02, -9.8810636e-02,
         4.6720497e-02,  3.0589908e-02, -3.9458768e-03, -3.1666238e-02,
        -4.5365624e-02, -2.9942218e-02, -5.8894847e-03,  1.6674181e-02,
        -1.5885459e-02,  2.1596789e-02,  2.9946296e-02, -4.9685515e-03,
         5.9288926e-03,  9.1029638e-03, -1.4521146e-02,  6.0366741e-03,
         3.2085737e-03, -9.0561126e-04,  2.3879191e-02, -8.7501118e-03,
        -4.4802050e-03, -7.8205504e-03, -2.9343322e-03,  1.4036173e-02,
         2.4955742e-02,  7.2905235e-03,  6.4317612e-03, -1.8830508e-02,
        -4.0564714e-03,  7.4658170e-03, -1.7083930e-02, -1.2393392e-02,
        -8.2753161e-03, -2.2886500e-03,  6.2091546e-03, -9.9342142e-04,
        -1.6771050e-02, -1.4426596e-02,  2.9852411e-03, -2.7077004e-02,
        -2.1504543e-03,  4.8811398e-03, -9.1972025e-03,  1.0233101e-02,
        -2.3461839e-02,  9.0730721e-03,  2.7200470e-02,  8.5940100e-03,

In [20]:
cur.close()

In [21]:
#probar que se cargaron los datos correctamente
cur = conn.cursor()
query_string = trunc.transform(vectorizer.transform(["perro"]))[0].tolist()
n_results = 3

query = "SELECT * FROM embeddings ORDER BY embedding <-> '{0}' LIMIT {1};".format(query_string, n_results)
results = cur.execute(query)

In [22]:
results.fetchone()


(60,
 'Benito Pérez Galdós',
 'Marianela',
 array([ 9.21555698e-01, -7.97646344e-02, -4.63331826e-02, -1.12572871e-02,
        -1.55452136e-02, -1.49548715e-02,  2.01366842e-03,  1.82374250e-02,
         5.76810585e-03, -1.49170272e-02, -1.57594569e-02,  9.12258681e-03,
         1.04457568e-02, -1.96398031e-02, -1.43525498e-02,  1.95458215e-02,
        -8.24103970e-03,  8.71512480e-03,  1.42907696e-02,  5.63304638e-04,
        -8.71146936e-03, -9.58764646e-03, -1.31970402e-02, -7.82990549e-03,
         3.67146172e-03, -1.34611432e-03, -1.56199106e-03, -3.65356961e-03,
        -1.04839895e-02,  4.87355748e-03, -1.44049833e-02, -5.29704383e-03,
         6.34220848e-03, -1.23889390e-02,  8.23387783e-03,  7.80418841e-03,
        -2.58644111e-03,  1.24405287e-02,  5.48031460e-03, -7.02808192e-03,
         4.09901980e-03,  5.07064816e-03,  3.48899746e-03,  2.74423137e-03,
         1.69265026e-03, -6.35176478e-03,  6.98194792e-03, -6.02693530e-03,
         2.28761174e-02,  1.09147988e-02, -9.

In [23]:
cur.close()

In [24]:
cur = conn.cursor()


query = "CREATE INDEX ON embeddings USING ivfflat (embedding vector_l2_ops) WITH (lists = 10);"

cur.execute(query)



<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=127.0.0.1 user=postgres database=vector) at 0x1d146bbc300>

In [25]:
#falta probarlo en la clase

In [26]:
import timeit
class DatabaseBenchmark:
    """
    Clase para benchmark de postgres
    Falta arreglar las querys para tipo python, decidir tipo de indexacion y ver como funciona con datos reales
    al contrario de chromi, no encuentro vectorizador por default, por lo que se puede utilizar tfidf
    """
    def __init__(self, db_connection, vectorizer, trunc, cost_per_hour = False):
        """
        db_connection: Conexión a base de datos
        vectorizer: text to vector vectorizer
        trunc: trunc svd from sparse matrix to lower dim matrix
        """

        self.db_connection = db_connection
        self.cost_per_hour = cost_per_hour
        self.vectorizer = vectorizer
        self.trunc = trunc

    def query(self, query_string, n_results = 1):
        """
        query_string: vector de representación de un texto
        n_results: numeros de vecinos a buscar
        """
        query_vector = self.trunc.transform(self.vectorizer.transform(query_string))[0].tolist()
        
        with db_connection.cursor() as cur:
            
            query = "SELECT * FROM items ORDER BY embedding <-> '{0}' LIMIT {1};".format(query_string, n_results) 
        
            results = cur.execute(query)

            return [record for record in results]  
#       

    def build_index(self, vectorcol):
        """
        incompleta, probar con datos
        """
        with db_connection.cursor() as cur:
        
            query = 'CREATE INDEX ON {0} USING hnsw (embedding vector_l2_ops);'.format(vectorcol)
        results = cur.execute(query)
#       

    def measure_qps(self, query_string, num_queries=5, n_results = 1):
        
        start_time = timeit.default_timer()
        for _ in range(num_queries):
            self.query(query_string= query_string, n_results= n_results)
        end_time = timeit.default_timer()
        
        qps = num_queries / (end_time - start_time)
        
        return qps
    
    def measure_qp_dollar(self, qps):
        if self.cost_per_hour is False:
            return "No hay costo por hora"
        qp_dollar = (qps / self.cost_per_hour) * 3600
        return qp_dollar
    
    def measure_latency(self, query_string, ):

        
        start_time = timeit.default_timer()
        self.query(query_string, 1)
        latency = timeit.default_timer() - start_time
        
        return latency
    
    def measure_index_building_time(self):

        
        start_time = timeit.default_timer()
        self.build_index(["Esto es un documento"], {"source": "my_source"}, ["id1"])
        index_building_time = timeit.default_timer() - start_time
        
        return index_building_time